# **Outils de cartographie: création de cartes intéractives à partir des lieux extraits de la Reconnaissance d'entitiés nommées**

3 outils pour 3 types de cartes interactives:
- carte regroupant tous les points de localisation d'un document, chapitre, paragraphe
- carte regroupant les points de localisation par groupes (et permet la sélection / dé-sélection
- carte créeant un trajet, parcours en prenant dans l'ordre les points de localisation

Fichiers d'entrée nécessaire: fichiet .txt avec liste des noms de lieux à identifier et géolocaliser

Sortie: cartes interactives (html) + possibilité de les enregistrer.

**Connexion à son compte GoogleDrive**:

- Lancer la cellule
- Cliquer sur le lien généré
- Copier / coller le mot de passe pour accorder l'accès au Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/My Drive/NER-demo/

Mounted at /content/drive/
/content/drive/My Drive/NER-demo


**Import des bibliothèques nécessaires à l'utilisation**:
- Lancer la cellule: les bibliothèques *Folium*, *Geopy* et *Panda* notamment sont importées.

In [3]:
!pip install geopy
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import csv

# **Outil 1: Création d'une carte avec tous les repères d'un fichier listant les lieux**

- Ligne 2: indiquer le nom du fichier .txt où sont listés les noms de lieux. Ici, le fichier se nomme: "loc_chap_7_bpt6k63947766.txt"

- Avant-dernière ligne: il est possible de modifier le nom du fichier html créée avec la carte. Ici, il se nomme "carte_chap_7_promenades.html".

- Pour régler la vue de la base de la carte: modifier la ligne "map_test = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)". Les deux nombres correspondent aux données latitude et longitude de Paris (car les lieux à afficher se trouvent à Paris), il est possible de changer avec n'importe quel lieu.

- Lancer la cellule: il est possible de voir s'afficher la liste de slieux, ainsi que les données de géolocalisation détectées pour chaque (la latitude et la longitude) qui vont permettre de placer les repères automatiquement sur la carte.




In [4]:
geolocator = Nominatim(user_agent="marinagiardinetti") ##Besoin d'un identifiant
with open ("/content/drive/My Drive/NER-demo/loc_chap_7_bpt6k63947766.txt", "r") as myfile:
    data=myfile.readlines()

df_coords={}
with open("output-demo.csv", "w") as my_coords:
  for lieu in data:
    try:
      location = geolocator.geocode(lieu, timeout=10)
      df_coords = pd.DataFrame.from_dict({
'place':[lieu],
'lat':[location.latitude],
'lon':[location.longitude],
})
      print(df_coords)
      df_coords.to_csv(my_coords, columns= ['place','lat', 'lon'], index = False, header=False)
    except AttributeError:
      print("Pas de géolocalisation trouvée")

with open('output-demo.csv',newline='') as f:
    r = csv.reader(f)
    data = [line for line in r]
with open('output-demo.csv','w',newline='') as f:
    w = csv.writer(f)
    w.writerow(['place', 'lat', 'lon'])
    w.writerows(data)

data_for_map = pd.read_csv("output-demo.csv", error_bad_lines=False, delimiter = ",")
print(data_for_map)

map_test = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)

def create_map_markers(row, map_name):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

found_locations = data_for_map[data_for_map['place']!= "Not Found"]
found_locations.apply(create_map_markers, map_name=map_test, axis='columns')
map_test.save("carte_chap_7_promenades.html")
map_test

                   place        lat       lon
0  Les tuileries paris\n  48.864244  2.330436
                place        lat       lon
0  Luxembourg paris\n  48.850433  2.332951
                    place        lat       lon
0  Champs-élysées paris\n  48.870757  2.305331
                   place        lat       lon
0  Parc monceaux paris\n  48.891389  2.313415
                        place        lat       lon
0  Jardin des plantes paris\n  48.843222  2.359509
                            place       lat       lon
0  Jardin du Palais Royal paris\n  48.86466  2.337631
                place        lat       lon
0  Bois de Boulogne\n  48.864093  2.248258
                 place        lat       lon
0  Bois de Vincennes\n  48.831622  2.434238
                 place        lat       lon
0  Montmartre  paris\n  48.886705  2.341572
                 place        lat       lon
0  Belleville  paris\n  48.871727  2.385085
                      place        lat       lon
0  Buttes Chaumont  paris\n

# **Outil 2: Création d'ue carte avec parcours interactif**



In [5]:
m = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
lines = [
    {
        "coordinates": [
            [2.330436, 48.864244],
            [2.332951, 48.850433],
        ],
        "dates": ["2017-06-02T00:00:00", "2017-06-02T00:10:00"],
        "color": "blue",
    },
    {
        "coordinates": [
            [2.332951, 48.850433],
            [2.305331, 48.870757],
        ],
        "dates": ["2017-06-02T00:10:00", "2017-06-02T00:20:00"],
        "color": "blue",
    },
    {
        "coordinates": [
            [2.305331, 48.870757],
            [2.313415, 48.891389],
        ],
        "dates": ["2017-06-02T00:20:00", "2017-06-02T00:30:00"],
        "color": "blue",
    },
    {
        "coordinates": [
            [2.313415, 48.891389],
            [2.359509, 48.843222],
        ],
        "dates": ["2017-06-02T00:30:00", "2017-06-02T00:40:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.359509, 48.843222],
            [2.337631, 48.864660],
        ],
        "dates": ["2017-06-02T00:40:00", "2017-06-02T00:50:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.337631, 48.864660],
            [2.248258, 48.864093],
        ],
        "dates": ["2017-06-02T00:50:00", "2017-06-02T01:00:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.248258, 48.864093],
            [2.434238, 48.831622],
        ],
        "dates": ["2017-06-02T01:10:00", "2017-06-02T01:20:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.434238, 48.831622],
            [2.341572, 48.886705],
        ],
        "dates": ["2017-06-02T01:20:00", "2017-06-02T01:30:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.341572, 48.886705],
            [2.385085, 48.871727],
        ],
        "dates": ["2017-06-02T01:30:00", "2017-06-02T01:40:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.385085, 48.871727],
            [2.381201, 48.878396],
        ],
        "dates": ["2017-06-02T01:40:00", "2017-06-02T01:50:00"],
        "color": "blue",
    },
        {
        "coordinates": [
            [2.381201, 48.878396],
            [2.337792, 48.821991],
        ],
        "dates": ["2017-06-02T01:50:00", "2017-06-02T02:00:00"],
        "color": "blue",
    },
]

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 5,
            },
        },
    }
    for line in lines
]

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)

m

# **Outil 3: Création d'une carte interactive avec groupes de lieux et sélection**

- Changer la vue de la base de carte
- Changer les noms des groupes / rajouter des groupes
- Changer les coordonnées gps des lieux de sélection

In [6]:
m = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)

fg = folium.FeatureGroup(name="Tous")
m.add_child(fg)

g1 = plugins.FeatureGroupSubGroup(fg, "Édifices civils et établissements publics")
m.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(fg, "Lieux de promenades")
m.add_child(g2)


folium.Marker([48.856426, 2.352528]).add_to(g1)
folium.Marker([48.850839, 2.326690]).add_to(g1)
folium.Marker([48.868630, 2.341474]).add_to(g1)
folium.Marker([48.854581, 2.340150]).add_to(g1)
folium.Marker([48.832397, 2.355583]).add_to(g1)


folium.Marker([48.864244, 2.330436]).add_to(g2)
folium.Marker([48.850433, 2.332951]).add_to(g2)
folium.Marker([48.870757, 2.305331]).add_to(g2)
folium.Marker([48.891389, 2.313415]).add_to(g2)
folium.Marker([48.843222, 2.359509]).add_to(g2)
folium.Marker([48.864660, 2.337631]).add_to(g2)
folium.Marker([48.864093, 2.248258]).add_to(g2)
folium.Marker([48.831622, 2.434238]).add_to(g2)
folium.Marker([48.886705, 2.341572]).add_to(g2)
folium.Marker([48.871727, 2.385085]).add_to(g2)
folium.Marker([48.878396, 2.381201]).add_to(g2)
folium.Marker([48.821991, 2.337792]).add_to(g2)

folium.LayerControl(collapsed=False).add_to(m)

m